In [2]:
import pandas as pd

In [1]:
fields = ["Time", "Honey_Level", "TicksInHive", "HasNectar", "TicksSincePollination", "CannotEnterHiveTicks", "CropsGrownSincePollination", "counter"]

In [4]:
def process_line(line: str, file_and_line_num: tuple[str, int] = (None, None)) -> dict:
    # filter out non-data rows
    if "Time: " not in line:
        return None
    
    fields_dict = {field: None for field in fields}
    idx_start = 0
    idx_end = 0
    
    for field in fields:
        # idx_start is the index at which the value starts for field; idx_end is the index at which the value ends with a new line
        idx_start = line.find(field + ": ", idx_start) + len(field + ": ")

        if idx_start == -1:
            continue

        idx_end = line.find(' ', idx_start)
        fields_dict[field] = int(line[idx_start : idx_end])
    
    fields_dict["file loc"] = file_and_line_num[0]
    fields_dict["line num"] = file_and_line_num[1]

    return fields_dict

### Testing Suite: 
Update the logs_to_check variable to include the log files you wish to test. Remember to rename the latest.log to something different so it doesn't get compressed automatically. 

In [34]:
logs_to_check = [
    "test-3.log"
]

dfs = []

for log_name in logs_to_check:
    processed_data = []
    line_num = 0
    with open(log_name, 'r') as log:
        for line in log:
            line_num += 1
            result = process_line(line, file_and_line_num=(log_name, line_num))
            if not result: # filter non-data rows
                continue
            processed_data.append(result) #df.loc[len(df.index)] = result # SUPER SLOW
    
    dfs.append(pd.DataFrame(processed_data))

working = pd.concat(dfs)
working

,Time,Honey_Level,TicksInHive,HasNectar,TicksSincePollination,CannotEnterHiveTicks,CropsGrownSincePollination,counter,file loc,line num
0,39814,0,0,0,0,0,0,2813,latest.log,59
1,39815,3,1924,0,0,0,0,2814,latest.log,61
2,39816,3,1925,0,1,0,0,2815,latest.log,62
3,39817,3,1926,0,2,0,0,2816,latest.log,63
4,39818,3,1927,0,1,0,0,2817,latest.log,64
...,...,...,...,...,...,...,...,...,...,...
771020,810834,1,2186,0,320,0,0,869,latest.log,771091
771021,810835,1,2187,0,321,0,0,870,latest.log,771092
771022,810836,1,2188,0,322,0,0,871,latest.log,771093
771023,810837,1,2189,0,323,0,0,872,latest.log,771094


In [33]:
working[working["counter"].diff(periods=-1) > 1].sort_values(by="counter", ascending=True)["counter"].describe()

count     209.000000
mean     2806.430622
std       185.463650
min      1059.000000
25%      2831.000000
50%      2833.000000
75%      2836.000000
max      2840.000000
Name: counter, dtype: float64

In [35]:
working[working["counter"].diff(periods=-1) > 1].sort_values(by="counter", ascending=True)["counter"].describe()

count     276.000000
mean     2800.594203
std       199.528704
min      1059.000000
25%      2832.000000
50%      2833.000000
75%      2836.000000
max      2840.000000
Name: counter, dtype: float64

In [27]:
working.loc[working["counter"].diff(periods=-1) == 1096]

,Time,Honey_Level,TicksInHive,HasNectar,TicksSincePollination,CannotEnterHiveTicks,CropsGrownSincePollination,counter,file loc,line num


# How Bees Work
We assume the bee is already associated to a flower and a nest. We do not cover: anger towards the player, crop fertilization, or finding a nest or flowers to associate with. 

While the bee is out of the nest, its TicksSincePollination increments. After a couple of ticks, it obtains nectar, setting HasNectar to 1. Only a few ticks later, it returns back into the nest for exactly 2400 ticks, exiting after time is up. Upon exit, the Honey_Level of the nest is incremented by either 1 (99% of the time) or 2 (1% of the time). The process repeats. 

### Possibility of 2-Increments
It was hypothesized from the source code (net/minecraft/world/level/block/entity/BeehiveBlockEntity.java) that when the Honey_Level if a nest/hive increases, it has a 1% chance of incrementing by 2 instead of the usual 1. The stats below show this is possible. 

In [179]:
# These were done on test-1.log and test-2.log
n_increments = working[(working["Honey_Level"].diff() != 0) & (working["Honey_Level"].diff() > 0)].shape[0] # exclude negative increments, since that's just the hive resetting
n_1_increments = working[working["Honey_Level"].diff() == 1].shape[0]
n_2_increments = working[working["Honey_Level"].diff() == 2].shape[0]

print(
    "Number of Increments:", n_increments, "\n",
    "Increments of 1:", n_1_increments, "\n",
    "Increments of 2:", n_2_increments, "\n",
    "--- Proportion of 2's:", round(n_2_increments / n_increments * 100, 2), "%"
)

working[working["Honey_Level"].diff() == 2]
#increments_plot = pd.DataFrame([{"Increments of 1": n_1_increments, "Increments of 2": n_2_increments}]).plot(kind="pie")

Number of Increments: 755 
 Increments of 1: 751 
 Increments of 2: 4 
 --- Proportion of 2's: 0.53 %


,Time,Honey_Level,TicksInHive,HasNectar,TicksSincePollination,CannotEnterHiveTicks,CropsGrownSincePollination,file loc,line num
110091,110097,4,0,0,0,0,0,test-1.log,110189
54211,166569,5,0,0,0,0,0,test-2.log,54275
291195,403553,5,0,0,0,0,0,test-2.log,291259
1210522,1322880,3,0,0,0,0,0,test-2.log,1210586


### Pollination Times (TicksSincePollination)
Below is the distribution of pollination times. Notice its minimum is 401. 

There appear to be some anomalies in the pollination times (measured by TicksSincePollination -- the time elapsed from the bee exiting the hive after making honey, to it returning to the hive). Most times seem normal with times ranging from mid-400s to low-400s, but about half of the pollination times appear to be preceded by a "buffer" of 2 to 20 ticks. This is worth noting, but for a TAS, these do not contribute towards finding an optimal lower bound for pollination times, so we will disregard them. If we wanted to incorporate the buffers, we'd have to add them to their following "real" pollination time. 

In [181]:
# -1 means that instead of comparing with previous, it compares with next. e.g. for 19, it usually does 19-18=1, but with periods=-1 it does 19-20=-1
mask = ~working["TicksSincePollination"].diff(periods=-1).isin([0, -1])
out_of_hive_durations_counts = working[mask]["TicksSincePollination"].value_counts().reset_index().sort_values("TicksSincePollination", ascending=False)
out_of_hive_durations_counts

,TicksSincePollination,count
41,457,1
40,450,1
34,445,2
39,444,1
44,443,1
46,442,1
42,440,1
30,439,2
38,438,1
24,435,5


In [183]:
working[mask][working[mask]["TicksSincePollination"] == 2] # to get file location to look at them first hand. modify number to heart's desire

,Time,Honey_Level,TicksInHive,HasNectar,TicksSincePollination,CannotEnterHiveTicks,CropsGrownSincePollination,file loc,line num
11340,11346,4,0,0,2,0,0,test-1.log,11432
70587,70593,1,0,0,2,0,0,test-1.log,70681
79039,79045,4,0,0,2,0,0,test-1.log,79133
14680,127038,2,0,0,2,0,0,test-2.log,14744
37261,149619,2,0,0,2,0,0,test-2.log,37325
...,...,...,...,...,...,...,...,...,...
1991766,2104124,4,0,0,2,0,0,test-2.log,1991830
2000227,2112585,3,0,0,2,0,0,test-2.log,2000291
2008678,2121036,2,0,0,2,0,0,test-2.log,2008742
2014324,2126682,4,0,0,2,0,0,test-2.log,2014388


In [184]:
ANOMALIES_THRESHOLD = 100
n_tickssincepollination_anomalies = working[mask][working[mask]["TicksSincePollination"] < ANOMALIES_THRESHOLD].shape[0]
n_tickssincepollination_typicals = working[mask][working[mask]["TicksSincePollination"] >= ANOMALIES_THRESHOLD].shape[0]
n_pollinations = n_tickssincepollination_typicals # assuming each anomaly is followed by a typical pollination counting

print(
    "Number of Pollinations:", n_pollinations, "\n",
    "Number of Typicals:", n_tickssincepollination_typicals, "\n",
    "Number of Anomalies:", n_tickssincepollination_anomalies, "\n",
    "--- Proportion of Anomalies:", round(n_tickssincepollination_anomalies / n_pollinations * 100, 2), "%"
)

Number of Pollinations: 756 
 Number of Typicals: 756 
 Number of Anomalies: 365 
 --- Proportion of Anomalies: 48.28 %


In [185]:
"""
HasNectar, length of runs
test-1.log, 518: 8
test-2.log, 384734: 0 but with anomaly of 10 ticks
test-2.log, 401649: 11
test-2.log, 1476095: 0, with no anomaly! ***
"""
working[mask][working[mask]["TicksSincePollination"] == 401] 


,Time,Honey_Level,TicksInHive,HasNectar,TicksSincePollination,CannotEnterHiveTicks,CropsGrownSincePollination,file loc,line num
430,436,0,0,0,401,0,0,test-1.log,518
384670,497028,1,0,0,401,0,0,test-2.log,384734
401585,513943,3,0,0,401,0,0,test-2.log,401649
1476031,1588389,1,0,0,401,0,0,test-2.log,1476095
